In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#plt.rcParams['figure.figsize'] = [15, 60]

taxon_name = "occs_all"
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
taxa = pd.read_csv(taxon_name+".csv", delimiter=",", 
    usecols=["ORDER", "FAMILY", "GENUS", "SPECIES", "MAX_AGE", "MIN_AGE", "LIDNUM","NAME", "BODYMASS", "DIET_1", "DIET_2", "DIET_3"], 
                   comment='#', na_values="\\N")
#taxa
#print(taxa)
print(taxa.isna().any())
taxa = taxa.fillna(-999)
print(taxa.isna().any())
print(taxa.shape)

indet_sp = taxa['SPECIES']=="indet." 
NA_sp = taxa['SPECIES']==-999 
NA_g = taxa['GENUS']==-999
NA_f = taxa['FAMILY']==-999
not_sp = taxa['SPECIES']=="sp."
indet_genus = taxa['GENUS']=="indet." 
#indet_genus = taxa['GENUS']=="Indet." 
not_genus = taxa['GENUS']=="gen." 
#not_genus = taxa['GENUS']=="Gen." 
inc_sedis_g = taxa['GENUS']=="incertae sedis"
inc_sedis_f = taxa['FAMILY']=="incertae sedis"
bad_min_age = taxa['MIN_AGE'] == -999
bad_max_age = taxa['MAX_AGE'] == -999
nomen_nudum_f = taxa['FAMILY'] == "nomen nudum"
nomen_nudum_g = taxa['GENUS'] == "nomen nudum"
Viverr = taxa['FAMILY']=="Viverravidae" 
bad_age = taxa['MAX_AGE'] > 70
not_ok = NA_g|NA_f| indet_genus | not_genus | inc_sedis_f| inc_sedis_g | bad_min_age | bad_max_age | bad_age| nomen_nudum_f | nomen_nudum_g
#Species data: indet_sp | not_sp | NA_sp|


taxa =taxa[~not_ok]
print("After cleanup:")
print(taxa.shape)

max_ma = np.array(taxa.MAX_AGE)
min_ma = np.array(taxa.MIN_AGE).astype(float)

age_list = np.mean([max_ma, min_ma], axis=0)

name = taxa.GENUS+" "+taxa.SPECIES
name_list = list(name)
#age_list = np.asarray(taxa['mean_ma'])




In [30]:
prob=taxa[taxa['DIET_1']=="herbivore"]

print("NOW herb:")
print(prob.shape)

max_ma = np.array(prob.MAX_AGE)
min_ma = np.array(prob.MIN_AGE).astype(float)

age_list = np.mean([max_ma, min_ma], axis=0)


#PyRate input format: Taxon_name 	Status 	MinAge 	MaxAge Site
#Pantheria DB
pantheria = pd.read_csv("../../Methods/PyRate-master/data/PanTHERIA_1-0_WR05_Aug2008.txt", delimiter="\t", 
    usecols=["MSW05_Binomial"], comment='#', na_values="\\N")
#print(pantheria.MSW05_Binomial)
name = prob.GENUS+"_"+prob.SPECIES
name_list = list(name)
pantheria_list = list(pantheria.MSW05_Binomial)
status = []
print(len(pantheria.MSW05_Binomial))
check_list = list(prob.GENUS+" "+prob.SPECIES)
for i in range(0, len(check_list)):
    #print(i)
    if check_list[i] in pantheria_list:
        print(check_list[i])
        status.append("Extant")
    else:
        status.append("Extinct")

pyRate_data = {"Taxon_name":name_list, "Status": status, "MinAge":min_ma, "MaxAge":list(prob.MAX_AGE), "Site":list(prob.LIDNUM)}
pyRateData = pd.DataFrame(pyRate_data)
pyRateData.to_csv("PyRate/NOW_carnivore.csv", index=False)



NOW herb:
(0, 12)
5416


In [ ]:
strat_stages = pd.read_csv("strat_stages.csv", delimiter=",", usecols=["stage", "bottom", "mid", "top", "stg"])
ind = np.digitize(age_list, strat_stages.top, right=True)
stages = strat_stages.stg[ind]
stage_names = strat_stages.stage[ind]

stage_step = 3
my_stages_bottom = np.round(np.linspace(66, 3, int(np.round(63./stage_step, 0))), 2) 
my_stages_mid = my_stages_bottom-stage_step/2.
my_stages_top = my_stages_bottom-stage_step
my_ind = np.digitize(age_list, my_stages_top, right=True)
stage_list = np.linspace(1, int(np.round((66 - 3.)/stage_step, 0)+1), int(np.round(63./stage_step, 0)))
print(stage_list)
print(my_stages_bottom)
my_stage_numbers = (stage_list[my_ind])

data = {'Name':name_list, "order":list(taxa.ORDER), "genus":list(taxa.GENUS),
        'stage':stage_names, 'stg':stages, "NOW_max_ma":list(taxa.MAX_AGE),  "NOW_mid_ma": list(age_list),  
        "NOW_min_ma":min_ma, "max_age":list(strat_stages.bottom[ind]),"mid_age":list(strat_stages.mid[ind]),
        'min_ma':list(strat_stages.top[ind]), "ID":list(taxa.LIDNUM), "col":list(taxa.NAME), 
        "NOW_stg":list(my_stage_numbers.astype(int)), "Mass":list(taxa.BODYMASS), "Diet_1":list(taxa.DIET_1), "Diet_2":list(taxa.DIET_2), "Diet_3":list(taxa.DIET_3)}

my_strat_data = {"stage_no":list(stage_list.astype(int)), "stage_bottom":list(my_stages_bottom), 
              "stage_mid":list(my_stages_mid), "stage_top":list(my_stages_top)}


NOW_strat = pd.DataFrame(my_strat_data)
NOW_strat.to_csv("NOW_strat.csv", index=False)

occ_data = pd.DataFrame(data)
occ_data.to_csv("occ_strat.csv", index=False, quoting=2)




#plt.rcParams['figure.figsize'] = [36, 72]
#uncertainty = 0.01*taxa.MAX_AGE
#plt.hist(age_list)
#plt.plot(my_stage_numbers, taxa.mean_ma, yerr=uncertainty, xerr=None, fmt='none')
#ax = plt.gca()
#ax.tick_params(axis='x', labelrotation = 90)
#plt.show()

In [16]:
#PyRate input format: Taxon_name 	Status 	MinAge 	MaxAge Site
#Pantheria DB
pantheria = pd.read_csv("../../Methods/PyRate-master/data/PanTHERIA_1-0_WR05_Aug2008.txt", delimiter="\t", 
    usecols=["MSW05_Binomial"], comment='#', na_values="\\N")
#print(pantheria.MSW05_Binomial)
name = taxa.GENUS+"_"+taxa.SPECIES
name_list = list(name)
pantheria_list = list(pantheria.MSW05_Binomial)
status = []
print(len(pantheria.MSW05_Binomial))
check_list = list(taxa.GENUS+" "+taxa.SPECIES)
for i in range(0, len(check_list)):
    #print(i)
    if check_list[i] in pantheria_list:
        print(check_list[i])
        status.append("Extant")
    else:
        status.append("Extinct")

pyRate_data = {"Taxon_name":name_list, "Status": status, "MinAge":min_ma, "MaxAge":list(taxa.MAX_AGE), "Site":list(taxa.LIDNUM)}
pyRateData = pd.DataFrame(pyRate_data)
pyRateData.to_csv("pyRate_data.csv", index=False)


5416
Connochaetes taurinus
Capricornis sumatraensis
Kobus kob
Kobus kob
Beatragus hunteri
Kobus kob
Aepyceros melampus
Aepyceros melampus
Kobus leche
Aepyceros melampus
Aepyceros melampus
Kobus kob
Aepyceros melampus
Connochaetes taurinus
Kobus ellipsiprymnus
Aepyceros melampus
Aepyceros melampus
Alcelaphus caama
Antidorcas marsupialis
Connochaetes gnou
Connochaetes taurinus
Damaliscus lunatus
Hippotragus equinus
Aepyceros melampus
Alcelaphus caama
Antidorcas marsupialis
Connochaetes taurinus
Damaliscus lunatus
Kobus ellipsiprymnus
Oreotragus oreotragus
Pelea capreolus
Aepyceros melampus
Connochaetes taurinus
Kobus ellipsiprymnus
Kobus kob
Oryx gazella
Aepyceros melampus
Kobus kob
Kobus leche
Aepyceros melampus
Kobus leche
Aepyceros melampus
Aepyceros melampus
Kobus kob
Kobus leche
Aepyceros melampus
Kobus kob
Aepyceros melampus
Aepyceros melampus
Kobus kob
Aepyceros melampus
Aepyceros melampus
Aepyceros melampus
Kobus kob
Kobus leche
Aepyceros melampus
Kobus kob
Aepyceros melampus
Aep

Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Hippopotamus amphibius
Moschus moschiferus
Moschus moschiferus
Moschus moschiferus
Moschus moschiferus
Moschus moschiferus
Hylochoerus meinertzhageni
Hylochoerus meinertzhageni
Hylochoerus meinertzhageni
Hylochoerus meinertzhageni
Phacochoerus aethiopicus
Phacochoerus aethiopicus
Phacochoerus aethiopicus
Potamochoerus porcus
Potamochoerus porcus
Phacochoerus aethiopicus
Phacochoerus aethiopicus
Phacochoerus aethiopicus
Phacochoerus aethiopicus
Potamochoerus porcus
Potamochoerus porcus
Phac

Taxidea taxus
Conepatus chinga
Conepatus leuconotus
Spilogale putorius
Procyon lotor
Mephitis mephitis
Spilogale putorius
Mustela frenata
Spilogale putorius
Spilogale putorius
Aonyx capensis
Arctonyx collaris
Ailuropoda melanoleuca
Helarctos malayanus
Ursus thibetanus
Paradoxurus hermaphroditus
Arctonyx collaris
Ailuropoda melanoleuca
Ursus thibetanus
Paradoxurus hermaphroditus
Arctonyx collaris
Ailuropoda melanoleuca
Ursus thibetanus
Paguma larvata
Paradoxurus hermaphroditus
Arctonyx collaris
Ailuropoda melanoleuca
Ursus thibetanus
Paradoxurus hermaphroditus
Arctonyx collaris
Ursus thibetanus
Paradoxurus hermaphroditus
Arctonyx collaris
Ailuropoda melanoleuca
Ursus thibetanus
Arctonyx collaris
Ailuropoda melanoleuca
Helarctos malayanus
Ursus thibetanus
Arctonyx collaris
Ailuropoda melanoleuca
Helarctos malayanus
Ursus thibetanus
Paradoxurus hermaphroditus
Arctonyx collaris
Helarctos malayanus
Paradoxurus hermaphroditus
Arctonyx collaris
Ursus thibetanus
Arctonyx collaris
Ailuropoda me